First I imported my libraries along with my raw data

In [184]:
import pandas as pd
import numpy as np
import seaborn
import datacompy

In [185]:
data_demo = pd.read_csv('data_demo.csv',encoding='ISO-8859–1')
data_diet = pd.read_csv('data_diet.csv',encoding='ISO-8859–1')
data_exam = pd.read_csv('data_exam.csv',encoding='ISO-8859–1')
data_labs = pd.read_csv('data_labs.csv',encoding='ISO-8859–1')
data_meds = pd.read_csv('data_meds.csv',encoding='ISO-8859–1')
data_ques = pd.read_csv('data_ques.csv',encoding='ISO-8859–1')


In [186]:
data_demo

,SEQN,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDAGEMN,RIDRETH1,RIDRETH3,RIDEXMON,RIDEXAGM,...,DMDHREDU,DMDHRMAR,DMDHSEDU,WTINT2YR,WTMEC2YR,SDMVPSU,SDMVSTRA,INDHHIN2,INDFMIN2,INDFMPIR
0,73557,8,2,1,69,NaN,4,4,1.0,NaN,...,3.0,4.0,NaN,13281.237386,13481.042095,1,112,4.0,4.0,0.84
1,73558,8,2,1,54,NaN,3,3,1.0,NaN,...,3.0,1.0,1.0,23682.057386,24471.769625,1,108,7.0,7.0,1.78
2,73559,8,2,1,72,NaN,3,3,2.0,NaN,...,4.0,1.0,3.0,57214.803319,57193.285376,1,109,10.0,10.0,4.51
3,73560,8,2,1,9,NaN,3,3,1.0,119.0,...,3.0,1.0,4.0,55201.178592,55766.512438,2,109,9.0,9.0,2.52
4,73561,8,2,2,73,NaN,3,3,1.0,NaN,...,5.0,1.0,5.0,63709.667069,65541.871229,2,116,15.0,15.0,5.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10170,83727,8,2,1,26,NaN,2,2,2.0,NaN,...,3.0,NaN,NaN,27141.631824,28115.568221,1,104,77.0,77.0,NaN
10171,83728,8,2,2,2,24.0,1,1,2.0,24.0,...,3.0,1.0,3.0,15274.475360,16512.138781,1,107,8.0,8.0,1.95
10172,83729,8,2,2,42,NaN,4,4,2.0,NaN,...,5.0,3.0,NaN,24122.248900,26902.344381,1,104,7.0,7.0,3.66
10173,83730,8,2,1,7,NaN,2,2,1.0,84.0,...,4.0,1.0,3.0,25521.878546,26686.025573,2,109,6.0,6.0,1.05


In [187]:
data_diet

,SEQN,WTDRD1,WTDR2D,DR1DRSTZ,DR1EXMER,DRABF,DRDINT,DR1DBIH,DR1DAY,DR1LANG,...,DRD370QQ,DRD370R,DRD370RQ,DRD370S,DRD370SQ,DRD370T,DRD370TQ,DRD370U,DRD370UQ,DRD370V
0,73557,16888.327864,12930.890649,1,49.0,2.0,2.0,6.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,73558,17932.143865,12684.148869,1,59.0,2.0,2.0,4.0,1.0,1.0,...,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0
2,73559,59641.812930,39394.236709,1,49.0,2.0,2.0,18.0,6.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,73560,142203.069917,125966.366442,1,54.0,2.0,2.0,21.0,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,73561,59052.357033,39004.892993,1,63.0,2.0,2.0,18.0,1.0,1.0,...,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9808,83727,14146.781331,10520.302070,1,59.0,2.0,2.0,8.0,7.0,1.0,...,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0
9809,83728,38750.761193,51815.103262,1,25.0,2.0,2.0,9.0,5.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9810,83729,15272.505199,10401.781170,1,59.0,2.0,2.0,1.0,6.0,1.0,...,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0
9811,83730,0.000000,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [188]:
data_exam

,SEQN,PEASCST1,PEASCTM1,PEASCCT1,BPXCHR,BPAARM,BPACSZ,BPXPLS,BPXPULS,BPXPTY,...,CSXLEAOD,CSXSOAOD,CSXGRAOD,CSXONOD,CSXNGSOD,CSXSLTRT,CSXSLTRG,CSXNART,CSXNARG,CSAEFFRT
0,73557,1,620.0,NaN,NaN,1.0,4.0,86.0,1.0,1.0,...,2.0,1.0,1.0,1.0,4.0,62.0,1.0,NaN,NaN,1.0
1,73558,1,766.0,NaN,NaN,1.0,4.0,74.0,1.0,1.0,...,3.0,1.0,2.0,3.0,4.0,28.0,1.0,NaN,NaN,1.0
2,73559,1,665.0,NaN,NaN,1.0,4.0,68.0,1.0,1.0,...,2.0,1.0,2.0,3.0,4.0,49.0,1.0,NaN,NaN,3.0
3,73560,1,803.0,NaN,NaN,1.0,2.0,64.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,73561,1,949.0,NaN,NaN,1.0,3.0,92.0,1.0,1.0,...,3.0,1.0,4.0,3.0,4.0,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9808,83727,1,611.0,NaN,NaN,1.0,3.0,60.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9809,83728,1,124.0,NaN,110.0,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9810,83729,1,679.0,NaN,NaN,1.0,4.0,80.0,1.0,1.0,...,1.0,1.0,1.0,3.0,4.0,55.0,1.0,NaN,NaN,3.0
9811,83730,1,381.0,NaN,72.0,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [189]:
data_labs

,SEQN,URXUMA,URXUMS,URXUCR.x,URXCRS,URDACT,WTSAF2YR.x,LBXAPB,LBDAPBSI,LBXSAL,...,URXUTL,URDUTLLC,URXUTU,URDUTULC,URXUUR,URDUURLC,URXPREG,URXUAS,LBDB12,LBDB12SI
0,73557,4.3,4.3,39.0,3447.6,11.03,NaN,NaN,NaN,4.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,524.0,386.7
1,73558,153.0,153.0,50.0,4420.0,306.00,NaN,NaN,NaN,4.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,507.0,374.2
2,73559,11.9,11.9,113.0,9989.2,10.53,142196.890197,57.0,0.57,3.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,732.0,540.2
3,73560,16.0,16.0,76.0,6718.4,21.05,NaN,NaN,NaN,NaN,...,0.062,0.0,0.238,0.0,0.0071,0.0,NaN,3.83,NaN,NaN
4,73561,255.0,255.0,147.0,12994.8,173.47,142266.006548,92.0,0.92,4.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,225.0,166.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9808,83727,1.9,1.9,47.0,4154.8,4.04,67775.162806,88.0,0.88,4.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,720.0,531.4
9809,83728,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9810,83729,6.0,6.0,117.0,10342.8,5.13,NaN,NaN,NaN,4.1,...,0.135,0.0,0.132,0.0,0.0081,0.0,2.0,9.75,388.0,286.3
9811,83730,4.5,4.5,86.0,7602.4,5.23,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [190]:
data_meds

,SEQN,RXDUSE,RXDDRUG,RXDDRGID,RXQSEEN,RXDDAYS,RXDRSC1,RXDRSC2,RXDRSC3,RXDRSD1,RXDRSD2,RXDRSD3,RXDCOUNT
0,73557,1,99999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
1,73557,1,INSULIN,d00262,2.0,1460.0,E11,NaN,NaN,Type 2 diabetes mellitus,NaN,NaN,2.0
2,73558,1,GABAPENTIN,d03182,1.0,243.0,G25.81,NaN,NaN,Restless legs syndrome,NaN,NaN,4.0
3,73558,1,INSULIN GLARGINE,d04538,1.0,365.0,E11,NaN,NaN,Type 2 diabetes mellitus,NaN,NaN,4.0
4,73558,1,OLMESARTAN,d04801,1.0,14.0,E11.2,NaN,NaN,Type 2 diabetes mellitus with kidney complicat...,NaN,NaN,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20189,83727,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20190,83728,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20191,83729,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20192,83730,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [191]:
data_ques

,SEQN,ACD011A,ACD011B,ACD011C,ACD040,ACD110,ALQ101,ALQ110,ALQ120Q,ALQ120U,...,WHD080U,WHD080L,WHD110,WHD120,WHD130,WHD140,WHQ150,WHQ030M,WHQ500,WHQ520
0,73557,1.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,3.0,...,NaN,40.0,270.0,200.0,69.0,270.0,62.0,NaN,NaN,NaN
1,73558,1.0,NaN,NaN,NaN,NaN,1.0,NaN,7.0,1.0,...,NaN,NaN,240.0,250.0,72.0,250.0,25.0,NaN,NaN,NaN
2,73559,1.0,NaN,NaN,NaN,NaN,1.0,NaN,0.0,NaN,...,NaN,NaN,180.0,190.0,70.0,228.0,35.0,NaN,NaN,NaN
3,73560,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,3.0
4,73561,1.0,NaN,NaN,NaN,NaN,1.0,NaN,0.0,NaN,...,NaN,NaN,150.0,135.0,67.0,170.0,60.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10170,83727,NaN,NaN,NaN,3.0,NaN,1.0,NaN,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,150.0,26.0,NaN,NaN,NaN
10171,83728,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10172,83729,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,155.0,135.0,NaN,195.0,42.0,NaN,NaN,NaN
10173,83730,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [192]:
compare1 = datacompy.Compare(data_demo,data_diet,join_columns="SEQN")
print(compare1.report())

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns   Rows
0       df1       47  10175
1       df2      168   9813

Column Summary
--------------

Number of columns in common: 1
Number of columns in df1 but not in df2: 46
Number of columns in df2 but not in df1: 167

Row Summary
-----------

Matched on: seqn
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 9,813
Number of rows in df1 but not in df2: 362
Number of rows in df2 but not in df1: 0

Number of rows with some compared columns unequal: 0
Number of rows with all compared columns equal: 9,813

Column Comparison
-----------------

Number of columns compared with some values unequal: 0
Number of columns compared with all values equal: 1
Total number of values which compare unequal: 0

Sample Rows Only in df1 (First 10 Columns)
------------------------------------------

       seqn  sddsrvyr  ridstatr  riagendr  ridageyr  ridage

In [193]:
compare2 = datacompy.Compare(data_demo,data_exam,join_columns="SEQN")
print(compare2.report())

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns   Rows
0       df1       47  10175
1       df2      224   9813

Column Summary
--------------

Number of columns in common: 1
Number of columns in df1 but not in df2: 46
Number of columns in df2 but not in df1: 223

Row Summary
-----------

Matched on: seqn
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 9,813
Number of rows in df1 but not in df2: 362
Number of rows in df2 but not in df1: 0

Number of rows with some compared columns unequal: 0
Number of rows with all compared columns equal: 9,813

Column Comparison
-----------------

Number of columns compared with some values unequal: 0
Number of columns compared with all values equal: 1
Total number of values which compare unequal: 0

Sample Rows Only in df1 (First 10 Columns)
------------------------------------------

       seqn  sddsrvyr  ridstatr  riagendr  ridageyr  ridage

In [194]:
compare3 = datacompy.Compare(data_demo,data_labs,join_columns="SEQN")
print(compare3.report())

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns   Rows
0       df1       47  10175
1       df2      424   9813

Column Summary
--------------

Number of columns in common: 1
Number of columns in df1 but not in df2: 46
Number of columns in df2 but not in df1: 423

Row Summary
-----------

Matched on: seqn
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 9,813
Number of rows in df1 but not in df2: 362
Number of rows in df2 but not in df1: 0

Number of rows with some compared columns unequal: 0
Number of rows with all compared columns equal: 9,813

Column Comparison
-----------------

Number of columns compared with some values unequal: 0
Number of columns compared with all values equal: 1
Total number of values which compare unequal: 0

Sample Rows Only in df1 (First 10 Columns)
------------------------------------------

       seqn  sddsrvyr  ridstatr  riagendr  ridageyr  ridage

In [195]:
compare4 = datacompy.Compare(data_demo,data_meds,join_columns="SEQN")
print(compare4.report())

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns   Rows
0       df1       47  10175
1       df2       13  20194

Column Summary
--------------

Number of columns in common: 1
Number of columns in df1 but not in df2: 46
Number of columns in df2 but not in df1: 12

Row Summary
-----------

Matched on: seqn
Any duplicates on match values: Yes
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 10,175
Number of rows in df1 but not in df2: 0
Number of rows in df2 but not in df1: 10,019

Number of rows with some compared columns unequal: 0
Number of rows with all compared columns equal: 10,175

Column Comparison
-----------------

Number of columns compared with some values unequal: 0
Number of columns compared with all values equal: 1
Total number of values which compare unequal: 0

Sample Rows Only in df2 (First 10 Columns)
------------------------------------------

        seqn  rxduse                  rxddrug rxddrgid

In [196]:
compare5 = datacompy.Compare(data_demo,data_ques,join_columns="SEQN")
print(compare5.report())

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns   Rows
0       df1       47  10175
1       df2      953  10175

Column Summary
--------------

Number of columns in common: 1
Number of columns in df1 but not in df2: 46
Number of columns in df2 but not in df1: 952

Row Summary
-----------

Matched on: seqn
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 10,175
Number of rows in df1 but not in df2: 0
Number of rows in df2 but not in df1: 0

Number of rows with some compared columns unequal: 0
Number of rows with all compared columns equal: 10,175

Column Comparison
-----------------

Number of columns compared with some values unequal: 0
Number of columns compared with all values equal: 1
Total number of values which compare unequal: 0




In [205]:
df = pd.merge(data_demo,data_diet,how = 'inner',on = 'seqn')
df = pd.merge(data_exam,df,how = 'inner',on = 'seqn')
df = pd.merge(data_meds,df,how = 'inner',on = 'seqn')
df = pd.merge(data_ques,df,how = 'inner',on = 'seqn')
df = pd.merge(data_labs,df,how = 'inner',on = 'seqn')
df

,seqn,urxuma,urxums,urxucr.x,urxcrs,urdact,wtsaf2yr.x,lbxapb,lbdapbsi,lbxsal,...,drd370qq,drd370r,drd370rq,drd370s,drd370sq,drd370t,drd370tq,drd370u,drd370uq,drd370v
0,73557,4.3,4.3,39.0,3447.6,11.03,NaN,NaN,NaN,4.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,73557,4.3,4.3,39.0,3447.6,11.03,NaN,NaN,NaN,4.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,73558,153.0,153.0,50.0,4420.0,306.00,NaN,NaN,NaN,4.7,...,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0
3,73558,153.0,153.0,50.0,4420.0,306.00,NaN,NaN,NaN,4.7,...,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0
4,73558,153.0,153.0,50.0,4420.0,306.00,NaN,NaN,NaN,4.7,...,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19575,83727,1.9,1.9,47.0,4154.8,4.04,67775.162806,88.0,0.88,4.9,...,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0
19576,83728,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19577,83729,6.0,6.0,117.0,10342.8,5.13,NaN,NaN,NaN,4.1,...,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0
19578,83730,4.5,4.5,86.0,7602.4,5.23,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [206]:
df=df.drop_duplicates(subset=['seqn'])

In [207]:
df.drop(columns=['siaproxy','siaintrp','fialang','fiaproxy','sialang','fiaintrp','mialang','miaproxy','miaintrp','aialanga','dr1exmer',
                  'dr1day','dr1lang','dr1helpd','dr1mnrsp','dr1dbih'])

,seqn,urxuma,urxums,urxucr.x,urxcrs,urdact,wtsaf2yr.x,lbxapb,lbdapbsi,lbxsal,...,drd370qq,drd370r,drd370rq,drd370s,drd370sq,drd370t,drd370tq,drd370u,drd370uq,drd370v
0,73557,4.3,4.3,39.0,3447.6,11.03,NaN,NaN,NaN,4.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,73558,153.0,153.0,50.0,4420.0,306.00,NaN,NaN,NaN,4.7,...,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0
6,73559,11.9,11.9,113.0,9989.2,10.53,142196.890197,57.0,0.57,3.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,73560,16.0,16.0,76.0,6718.4,21.05,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,73561,255.0,255.0,147.0,12994.8,173.47,142266.006548,92.0,0.92,4.3,...,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19575,83727,1.9,1.9,47.0,4154.8,4.04,67775.162806,88.0,0.88,4.9,...,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0
19576,83728,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19577,83729,6.0,6.0,117.0,10342.8,5.13,NaN,NaN,NaN,4.1,...,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0
19578,83730,4.5,4.5,86.0,7602.4,5.23,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [208]:
df.filter(regex = 'drqsdt')

,drqsdt1,drqsdt2,drqsdt3,drqsdt4,drqsdt5,drqsdt6,drqsdt7,drqsdt8,drqsdt9,drqsdt10,drqsdt11,drqsdt12,drqsdt91
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19576,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19577,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19578,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [221]:
#diet = df.filter(regex = 'drqsdt')
#diet
#df['drqsdt91'] = df['drqsdt91'].map({ 91.0 : 'Other special diet' })
#df['drqsdt91'] 

df['drqsdt1']=df['drqsdt1'].replace(to_replace = 1.0, value = 'Weight Loss or Low Calorie Diet')
df['drqsdt2']=df['drqsdt2'].replace(to_replace = 2.0, value = 'Low Fat or Low Cholesterol Diet')
df['drqsdt3']=df['drqsdt3'].replace(to_replace = 3.0, value = 'Low Salt or Low Sodium Diet')
df['drqsdt4']=df['drqsdt4'].replace(to_replace = 4.0, value = 'Sugar Free or Low Sugar Diet')
df['drqsdt5']=df['drqsdt5'].replace(to_replace = 5.0, value  = 'Low Fiber Diet')
df['drqsdt6']=df['drqsdt6'].replace(to_replace=6.0,value = 'High Fiber Diet')
df['drqsdt7']=df['drqsdt7'].replace(to_replace=7.0,value = 'Diabetic Diet')
df['drqsdt8']=df['drqsdt8'].replace(to_replace=8.0,value= 'Weight Gain or Muscle Building Diet')
df['drqsdt9']=df['drqsdt9'].replace(to_replace= 9.0, value = 'Low Carbohydrate Diet')
df['drqsdt10']=df['drqsdt10'].replace(to_replace =10.0,value = 'High Protein Diet')
df['drqsdt11']=df['drqsdt11'].replace(to_replace =11.0,value = 'Gluten-Free or Celiac Diet')
df['drqsdt12']=df['drqsdt12'].replace(to_replace =12.0, value = 'Renal or Kidney Diet')
df['drqsdt91']=df['drqsdt91'].replace(to_replace =91.0, value ="Other Special Diet")
df.filter(regex = 'drqsdt')


C:\Users\nicol\AppData\Local\Temp/ipykernel_14836/2900502462.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['drqsdt1']=df['drqsdt1'].replace(to_replace = 1.0, value = 'Weight Loss or Low Calorie Diet')
C:\Users\nicol\AppData\Local\Temp/ipykernel_14836/2900502462.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['drqsdt2']=df['drqsdt2'].replace(to_replace = 2.0, value = 'Low Fat or Low Cholesterol Diet')
C:\Users\nicol\AppData\Local\Temp/ipykernel_14836/2900502462.py:8: SettingWithCopyWarning: 


,drqsdt1,drqsdt2,drqsdt3,drqsdt4,drqsdt5,drqsdt6,drqsdt7,drqsdt8,drqsdt9,drqsdt10,drqsdt11,drqsdt12,drqsdt91
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,Diabetic Diet,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other Special Diet
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19576,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19577,Weight Loss or Low Calorie Diet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19578,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [228]:
cols=df['drqsdt1','drqsdt2','drqsdt3','drqsdt5','drqsdt6','drqsdt7','drqsdt8','drqsdt9','drqsdt10','drqsdt11','drqsdt12','drqsdt91']
cols2=df['drqsdt2','drqsdt3','drqsdt5','drqsdt6','drqsdt7','drqsdt8','drqsdt9','drqsdt10','drqsdt11','drqsdt12','drqsdt91']
df.assign(df['drqsdt1']=df[cols].sum(1)).drop(cols2, 1)


SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (Temp/ipykernel_14836/1321185538.py, line 3)